In [1]:
import torchtext
from torchtext import data
from torchtext.data import Field
import pandas as pd
import numpy as np
import spacy
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,wordpunct_tokenize
import string
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
nlp = spacy.load('en')

In [3]:
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED)

In [4]:
stopwords = set(stopwords.words('english'))

In [5]:
stopwords.add("'s")
stopwords.add("...")

In [6]:
def text_preprocessing(x):
    x = str(x)
    
    #cleaning url, @name, emojis, smiley etc
    text = p.clean(x)
    
    #punctation removal
    no_punct = [word for word in word_tokenize(text) if word not in string.punctuation]
    
    #stopword removal
    no_sw = set(wrd for wrd in no_punct if wrd not in stopwords)
    
    return " ".join(w for w in no_sw if w).strip()

### Creating Dataset using TabularDataset from Torchtext

In [55]:
TEXT = Field(tokenize='spacy')#, preprocessing = data.Pipeline(lambda x : text_preprocessing(x)))

LABEL = data.LabelField(dtype = torch.float)

tweet = data.TabularDataset(path='train_original.csv', format='csv',
                            fields={
                                'text': ('text', TEXT),
                                'target': ('labels', LABEL)
                                   }
                           )

In [56]:
TEXT.build_vocab(tweet, max_size=10000)

In [57]:
LABEL.build_vocab(tweet)

In [58]:
LABEL.vocab.freqs

Counter({'1': 3271, '0': 4342})

In [61]:
#TEXT.vocab.freqs.pop('') # remove empty string

In [62]:
train, valid = tweet.split(split_ratio=0.8)

### Convert Dataset into Iterator 

In [63]:
device = torch.device('cuda')

### BucketIterator - 

>**Shuffles the data in iterator**

>**It is recommended that for test data we should use just itertor, since we are evaluating the model.**

In [64]:
train_iter, val_iter, test_iter = data.BucketIterator.splits((train, valid, None),batch_sizes=(64, 128, 0),
                                                            sort_key=lambda x: len(x.text), device=device)

In [65]:
type(train_iter.data()[0].text)

list

In [66]:
type(val_iter.data()[99].text)

list

In [67]:
test = pd.read_csv("data/test.csv")

In [68]:
%%time
test = data.TabularDataset(path='data/test.csv', format='csv',
                            fields={
                                'text': ('text', TEXT),
                                }
                           )

CPU times: user 475 ms, sys: 135 µs, total: 475 ms
Wall time: 475 ms


In [69]:
test_iter = data.Iterator(dataset=test, device=device, batch_size=128, sort=False)

## Model Building

In [70]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))


In [71]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 128
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [72]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 523,269 trainable parameters


In [73]:
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [74]:
criterion = nn.BCEWithLogitsLoss()

In [75]:
model = model.to(device)
criterion = criterion.to(device)

In [76]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [77]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.labels)
        
        acc = binary_accuracy(predictions, batch.labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [78]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.labels)
            
            acc = binary_accuracy(predictions, batch.labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [79]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [80]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.686 | Train Acc: 55.80%
	 Val. Loss: 0.681 |  Val. Acc: 57.57%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.686 | Train Acc: 55.88%
	 Val. Loss: 0.682 |  Val. Acc: 57.57%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.686 | Train Acc: 55.75%
	 Val. Loss: 0.681 |  Val. Acc: 57.70%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.686 | Train Acc: 55.92%
	 Val. Loss: 0.681 |  Val. Acc: 57.96%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.687 | Train Acc: 55.55%
	 Val. Loss: 0.681 |  Val. Acc: 58.03%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.687 | Train Acc: 55.50%
	 Val. Loss: 0.681 |  Val. Acc: 58.16%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.687 | Train Acc: 55.89%
	 Val. Loss: 0.681 |  Val. Acc: 58.09%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.686 | Train Acc: 55.72%
	 Val. Loss: 0.681 |  Val. Acc: 57.83%
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.687 | Train Acc: 55.38%
	 Val. Loss: 0.681 |  Val. Acc: 57.83%
Epoch: 10 | Epoch Time: 0m 0